Introdução à Pyspark

ATIVIDADE 1

In [0]:
# Importação das bibliotecas principais
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Criando uma sessão Spark
spark = SparkSession.builder.appName("Atividade1_PySpark").getOrCreate()




Exercício 1.1

In [0]:
fut_players_data = spark.read.table("workspace.default.fut_players_data") 


# Exibindo o schema e as primeiras linhas
# fut_players_data.printSchema()
display(fut_players_data.limit(5))



Atividade 1.2

In [0]:
# Filtro para retornar os jogadores the bests
the_best = (fut_players_data
            .filter((F.col("dribbling") > 90) & (F.col("shooting") > 90))
            .select("player_id", "player_name", "position", "dribbling", "shooting", "overall")
           )

# nationalities é um DataFrame da nacionalidade dos jogadores
nationalities = (fut_players_data
                 .select("player_id", "player_name", "nationality")
                )

# Faça um join dos dois DataFrames, mantendo todos os jogadores de the_best e obtendo suas nacionalidades
the_best_nationality = (the_best # Dataframe esquerdo
                        .join(nationalities, # Dataframe direito
                              on=["player_id", "player_name"], # Duas correspondencias para não repetir player name
                              how="left")
                       )

# Exibir o resultado final
display(the_best_nationality)

Exercício 2

In [0]:
pkmn = spark.read.table("workspace.default.pokemon_data")
display(pkmn.limit(5))

In [0]:

pkmn = pkmn.withColumn("Sum_Attack_Speed", F.col("Attack") + F.col("Speed"))
display(pkmn.limit(5))

In [0]:
# Conferindo os tipos únicos antes da alteração
pkmn.filter(pkmn["Speed"] > 100).select("Type 1").distinct().display()


# 4️⃣ Alterar os Pokémons com Speed > 100 para Type 1 = "Fire"
pkmn = pkmn.withColumn(
    "Type 1",
    F.when(F.col("Speed") > 100, "Fire").otherwise(F.col("Type 1"))
)

# 5️⃣ Conferir os tipos únicos DEPOIS da alteração
pkmn.filter(pkmn["Speed"] > 100).select("Type 1").distinct().display()


In [0]:
pkmn = (
    pkmn
    .withColumnRenamed("Type 1", "Type_1")
    .withColumnRenamed("Type 2", "Type_2")
    .withColumnRenamed("Sp. Atk", "Sp_Atk")
    .withColumnRenamed("Sp. Def", "Sp_Def")
)


Exercicio 3

In [0]:
pkmn_soma = (
    pkmn
    .groupBy("Generation")  # Agrupar por geração
    .agg(
        F.sum(F.col("Legendary").cast("int"))  # Converter True/False em 1/0 e somar
        .alias("Qtd_Legendary")                # Nomear nova coluna
    )
)

# Exibir resultado
display(pkmn_soma)

In [0]:
pkmn_media = (
    pkmn
    .groupBy("Type_1")  # Agrupar por tipo principal 
    .agg(
        F.mean("HP").alias("HP_medio"),
        F.mean("Attack").alias("Attack_medio"),
        F.mean("Defense").alias("Defense_medio")
    )
)

# Exibir resultado
display(pkmn_media)

ATIVIDADE 2

Exercício 1

In [0]:
# Agrupar por nacionalidade e calcular a média do overall
country_avg_overall = (
    fut_players_data
    .groupBy("nationality")                  # Agrupar por país
    .agg(F.mean("overall").alias("avg_overall"))  # Calcular média e renomear
)

# Exibir o DataFrame com as médias
display(country_avg_overall)

In [0]:
# Retornar a nacionalidade com maior overall médio e o overall médio do Brasil
melhor = (
    country_avg_overall
    .orderBy(F.col("avg_overall").desc())
    .limit(1)
    .collect()[0]
)

brasil = (
    country_avg_overall
    .filter(F.col("nationality") == "Brazil")
    .collect()[0]
)

display({
    "Melhor overall médio": f"{melhor['nationality']}: {melhor['avg_overall']:.2f}",
    "Overall médio do Brasil": round(brasil['avg_overall'], 2)
})


Exercício 2.1

In [0]:
fut_players = (
    fut_players_data
    .select('player_id', 'overall')  # Seleciona colunas relevantes
    .withColumn(
        'classification',
        F.when(F.col('overall') <= 50, 'Amador')
         .when((F.col('overall') >= 51) & (F.col('overall') <= 60), 'Ruim')
         .when((F.col('overall') >= 61) & (F.col('overall') <= 70), 'Ok')
         .when((F.col('overall') >= 71) & (F.col('overall') <= 80), 'Bom')
         .when((F.col('overall') >= 81) & (F.col('overall') <= 90), 'Ótimo')
         .otherwise('Lenda')  
    )
)

# Contar quantos jogadores há em cada classificação
fut_players.groupBy("classification").count().orderBy("count", ascending=False).display()

DESAFIO

In [0]:
fut = spark.table("workspace.default.fut_players_data") #Carregar base novamento

# Filtro pelos jogadores brasileiros
br_players = fut.filter(F.col("nationality") == "Brazil")

In [0]:
# Criando coluna do grupo das posições
br_players = br_players.withColumn(
    "position_group",
    F.when(F.col("position") == "GK", "Goleiro")
     .when(F.col("position").isin("CB", "LB", "RB", "LWB", "RWB"), "Defesa")
     .when(F.col("position").isin("CM", "CDM", "CAM", "LM", "RM"), "Meio")
     .when(F.col("position").isin("ST", "CF", "LW", "RW", "LF", "RF"), "Ataque")
     .otherwise("Outros")
)

In [0]:
# Cria uma janela que divide o DataFrame em grupos por "position_group"
windowSpec = Window.partitionBy("position_group").orderBy(F.col("overall").desc())

In [0]:
# Cria a coluna 'rank' que numera os jogadores por grupo e filtra os melhores de cada posição
br_best = (
    br_players
    .withColumn("rank", F.row_number().over(windowSpec))
    .filter(
        ((F.col("position_group") == "Goleiro") & (F.col("rank") <= 1)) |
        ((F.col("position_group") == "Defesa") & (F.col("rank") <= 4)) |
        ((F.col("position_group") == "Meio") & (F.col("rank") <= 4)) |
        ((F.col("position_group") == "Ataque") & (F.col("rank") <= 2))
    )
    .select("nationality", "position_group", "player_name", "overall")
    .orderBy(F.col("position_group"))
)

display(br_best)

BONUS

In [0]:
#Filtro de jogadores brasileiros agrupando por player name (evitar duplicação)
br_players = (
    fut.filter(F.col("nationality") == "Brazil")
       .groupBy("player_name")
       .agg(
           F.max("overall").alias("overall"),
           F.first("position").alias("position"),
           F.first("nationality").alias("nationality")
       )
)

In [0]:
br_players = br_players.withColumn(
    "position_group",
    F.when(F.col("position") == "GK", "Goleiro")
     .when(F.col("position").isin("CB", "LB", "RB", "LWB", "RWB"), "Defesa")
     .when(F.col("position").isin("CM", "CDM", "CAM", "LM", "RM"), "Meio")
     .when(F.col("position").isin("ST", "CF", "LW", "RW", "LF", "RF"), "Ataque")
     .otherwise("Outros")
)

In [0]:
windowSpec = Window.partitionBy("position_group").orderBy(F.col("overall").desc())

In [0]:
br_best = (
    br_players
    .withColumn("rank", F.row_number().over(windowSpec))
    .filter(
        ((F.col("position_group") == "Goleiro") & (F.col("rank") <= 1)) |
        ((F.col("position_group") == "Defesa") & (F.col("rank") <= 4)) |
        ((F.col("position_group") == "Meio") & (F.col("rank") <= 4)) |
        ((F.col("position_group") == "Ataque") & (F.col("rank") <= 2))
    )
    .select("nationality", "position_group", "player_name", "overall")
    .orderBy(F.col("position_group"))
)

display(br_best)